In [ ]:
import os 
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
     

In [18]:
from urllib.request import urlretrieve

In [19]:
files = [
    "https://www.census.gov/content/dam/Census/library/publications/2022/demo/p70-178.pdf",
    "https://www.census.gov/content/dam/Census/library/publications/2023/acs/acsbr-017.pdf",
    "https://www.census.gov/content/dam/Census/library/publications/2023/acs/acsbr-016.pdf",
    "https://www.census.gov/content/dam/Census/library/publications/2023/acs/acsbr-015.pdf",
]
os.makedirs("us_census", exist_ok=True)

In [20]:
for url in files:
    file_path = os.path.join("us_census", url.rpartition("/")[2])
    urlretrieve(url, file_path)

In [21]:
loader = PyPDFDirectoryLoader("./us_census/")

In [22]:
docs_before_split = loader.load()

In [23]:
docs_before_split[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.2 (Windows)', 'creationdate': '2023-09-09T07:52:17-04:00', 'author': 'U.S. Census Bureau', 'keywords': 'acsbr-015', 'moddate': '2023-09-12T14:44:47+01:00', 'title': 'Health Insurance Coverage Status and Type by Geography: 2021 and 2022', 'trapped': '/false', 'source': 'us_census\\acsbr-015.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.1 Public policy changes included \nthe renewal of the Public Health Emergency, wh

In [8]:
text_splitter =  RecursiveCharacterTextSplitter(
    chunk_size =700,
    chunk_overlap = 50
)
docs_after_split = text_splitter.split_documents(docs_before_split)


In [9]:
len(docs_after_split[0].page_content)

694

In [10]:
avg_doc_length = lambda docs: sum([len(doc.page_content) for doc in docs])//len(docs)

In [11]:
avg_char_before_split = avg_doc_length(docs_before_split)
avg_char_after_split = avg_doc_length(docs_after_split)

In [12]:
print(f'before split: {avg_char_before_split}')
print(f'after split: {avg_char_after_split}')

before split: 3840
after split: 624


In [ ]:
from langchain_mistralai import MistralAIEmbeddings

embeddings = MistralAIEmbeddings(
    model="mistral-embed",
)

In [75]:
vectorstore = FAISS.from_documents(docs_after_split, embeddings)

In [76]:
retriever = vectorstore.as_retriever(search_type="similarity" , search_kwargs={"k" : 3})

In [93]:
relevants = vectorstore.similarity_search("what are the trends in median household income across different states in the us between 2021 and 2022?")

In [98]:
text = []
for document in relevants:
    text.append(document.page_content)
for line in text:
    print(line)
    print("-------------------")

hold income in 2022 was $24,112 
(Table 1 and Figure 2). Median 
household income was lower than 
the U.S. median in 30 states and 
Puerto Rico. It was higher than the 
U.S. median in 17 states and the 
District of Columbia. The medians 
for Arizona, Oregon, and Vermont 
were not statistically different from 
the U.S. median.
From 2021 to 2022, five states—
Alabama, Alaska, Delaware, Florida, 
and Utah—showed a statistically 
significant increase in real median 
household income; 17 states 
showed a decrease. Real median 
household income in 2022 was not 
statistically different from that in 
2021 for 28 states, the District of 
Columbia, and Puerto Rico  
(Table 1).
-------------------
2006 (the earliest year for which 
the Gini Index is available) to 2022, 
the index increased 4.7 percent, 
from 0.464 to 0.486.
SUMMARY
The U.S. median household income 
declined 0.8 percent to $74,755 
from 2021 to 2022. Between 2021 
and 2022, five states—Alabama, 
Alaska, Delaware, Florida, and 
Uta

In [77]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
)

In [79]:
messages = [
    (
        "system",
        "You are a helpful assistant.",
    ),
    ("human", "Hello"),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="Hello! How can I assist you today? Let's have a friendly conversation.  How are you doing?", additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 12, 'total_tokens': 36, 'completion_tokens': 24}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-703ca63b-8a9c-490f-b397-dc7f43f4b513-0', usage_metadata={'input_tokens': 12, 'output_tokens': 24, 'total_tokens': 36})

In [80]:
prompt_template = """Use the following pieces of context to answer the question at the end. Please follow the following rules:
1. If you don't know the answer, don't try to make up an answer. Just say "I can't find the final answer but you may want to check the following links".
2. If you find the answer, write the answer in a concise way with five sentences maximum.

{context}

Question: {question}

Helpful Answer:
"""

PROMPT = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)
     

In [81]:
retrievalQA = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs = {"prompt" : PROMPT}
)

### USE THE RAG !!! 

In [90]:
result = retrievalQA.invoke({"query" : "in which year : Health Insurance Coverage Status and Type by Geography"})

In [92]:
result['result']

'The document discusses health insurance coverage status and type by geography for the years 2021 and 2022.'

In [85]:
print(result.keys())

dict_keys(['query', 'result', 'source_documents'])


In [86]:
relevant_docs = result['source_documents']
print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')
print("*" * 100)
for i, doc in enumerate(relevant_docs):
    print(f"Relevant Document #{i+1}:\nSource file: {doc.metadata['source']}, Page: {doc.metadata['page']}\nContent: {doc.page_content}")
    print("-"*100)
    print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')

There are 3 documents retrieved which are relevant to the query.
****************************************************************************************************
Relevant Document #1:
Source file: us_census\acsbr-016.pdf, Page: 1
Content: /one.tab/six.tab
/one.tab/eight.tab
/two.tab/zero.tab/two.tab/two.tab ‘/two.tab/one.tab‘/two.tab/zero.tab‘/one.tab/nine.tab ‘/one.tab/eight.tab‘/one.tab/seven.tab‘/one.tab/six.tab ‘/one.tab/five.tab ‘/one.tab/four.tab ‘/one.tab/three.tab ‘/one.tab/two.tab ‘/one.tab/one.tab ‘/one.tab/zero.tab ‘/zero.tab/nine.tab ‘/zero.tab/eight.tab ‘/zero.tab/seven.tab ‘/zero.tab/six.tab /two.tab/zero.tab/zero.tab/five.tab
----------------------------------------------------------------------------------------------------
There are 3 documents retrieved which are relevant to the query.
Relevant Document #2:
Source file: us_census\acsbr-017.pdf, Page: 1
Content: /six.tab/five.tab
/seven.tab/zero.tab
/seven.tab/five.tab
/eight.tab/zero.tab
/two.tab/zero.tab/two.tab/